In [1]:
import pandas as pd


In [2]:
df=pd.read_json("/home/miguel/Documents/Faculdade/PRI/feup-pri/0.json")


In [3]:
df["text"].str.replace("[\n\t\r]"," ")
df["text"].str.replace(" +"," ")


/tmp/ipykernel_17634/2193786047.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"].str.replace("[\n\t\r]"," ")
/tmp/ipykernel_17634/2193786047.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"].str.replace(" +"," ")


0        1 ESTATUTOS DO PARTIDO SOCIALISTA CAPÍTULO ...
1      1 Índice INTRODUÇÃO ............................
2     PROGRAMA ELEITORAL DO PARTIDO SOCIALISTA PROGR...
3     partido socialista DECLARAÇÃO DE PRINCÍPIOS oU...
4        1 ESTATUTOS DO PARTIDO SOCIALISTA CAPÍTULO ...
5     PROGRAMA ELEITORAL DO PARTIDO SOCIALISTA PROGR...
6     PROGRAMA ELEITORAL DO PARTIDO SOCIALISTA PROGR...
7      Grupo de trabalho Políticas de Habitação, Cré...
8          FEDERAÇÃO DISTRITAL DA ÁREA URBANA DE LIS...
9        FEDERAÇÃO DISTRITAL DE VIANA DO CASTELO  Ve...
10       FEDERAÇÃO DISTRITAL DE VISEU  Ver verso da ...
11       FEDERAÇÃO DISTRITAL DE COIMBRA  Ver verso d...
12         FEDERAÇÃO DISTRITAL DA ÁREA URBANA DE LIS...
13       FEDERAÇÃO DISTRITAL DE CASTELO BRANCO  Ver ...
14       FEDERAÇÃO DISTRITAL DE LEIRIA  Ver verso da...
15       FEDERAÇÃO DISTRITAL DE AVEIRO  Ver verso da...
16       FEDERAÇÃO DISTRITAL DE VILA REAL  Ver verso...
17       FEDERAÇÃO REGIONAL DO OESTE  Ver verso 

In [4]:

df.drop_duplicates(subset="text",
                     keep="first", inplace=True)


,date,link,contentLength,text
0,2020-03-06 13:39:44,https://arquivo.pt/wayback/20200306133944/http...,98938,1 ESTATUTOS DO PARTIDO SOCIALISTA CAPÍTULO ...
1,2020-05-05 19:41:55,https://arquivo.pt/wayback/20200505194155/http...,468360,1 Índice INTRODUÇÃO ............................
2,2020-03-06 13:35:53,https://arquivo.pt/wayback/20200306133553/http...,507721,PROGRAMA ELEITORAL DO PARTIDO SOCIALISTA PROGR...
3,2020-06-29 01:31:57,https://arquivo.pt/wayback/20200629013157/http...,37388,partido socialista DECLARAÇÃO DE PRINCÍPIOS oU...
7,2020-06-29 01:39:02,https://arquivo.pt/wayback/20200629013902/http...,34032,"Grupo de trabalho Políticas de Habitação, Cré..."
8,2020-09-03 15:51:42,https://arquivo.pt/wayback/20200903155142/http...,2268,FEDERAÇÃO DISTRITAL DA ÁREA URBANA DE LIS...
9,2020-09-03 15:52:24,https://arquivo.pt/wayback/20200903155224/http...,3317,FEDERAÇÃO DISTRITAL DE VIANA DO CASTELO Ve...
10,2020-09-03 15:52:33,https://arquivo.pt/wayback/20200903155233/http...,4280,FEDERAÇÃO DISTRITAL DE VISEU Ver verso da ...
11,2020-09-03 15:51:28,https://arquivo.pt/wayback/20200903155128/http...,2358,FEDERAÇÃO DISTRITAL DE COIMBRA Ver verso d...
12,2020-09-03 15:51:46,https://arquivo.pt/wayback/20200903155146/http...,2165,FEDERAÇÃO DISTRITAL DA ÁREA URBANA DE LIS...
